# Visualize trace divergence and latency patterns

This notebook focuses on **visual, exploratory views** of instability signals:

- per-trace latency curves over time,
- highlighting drift-like vs correction events,
- scanning for large relative-latency gaps in long-horizon traces.

It expects a JSONL file generated from:

```bash
python examples/synthetic_traces/generate_synthetic_traces.py \
  --variant long_horizon --sessions 5 --turns 40 \
  > examples/synthetic_traces/long_horizon_traces.jsonl
```


In [ ]:
import json
from collections import defaultdict
from dataclasses import dataclass
from pathlib import Path
from typing import Any, Dict, List, Optional

import matplotlib.pyplot as plt

DATA_DIR = Path("examples") / "synthetic_traces"
LONG_HORIZON_FILE = DATA_DIR / "long_horizon_traces.jsonl"
print(f"Expected long-horizon file: {LONG_HORIZON_FILE.resolve()}")


In [ ]:
@dataclass
class Event:
    raw: Dict[str, Any]

    @property
    def trace_id(self) -> str:
        return str(self.raw.get("trace_id", ""))

    @property
    def event_type(self) -> str:
        return str(self.raw.get("event_type", ""))

    @property
    def payload(self) -> Dict[str, Any]:
        obj = self.raw.get("payload")
        return obj if isinstance(obj, dict) else {}

    @property
    def latency_ms(self) -> Optional[float]:
        val = self.payload.get("latency_ms")
        try:
            return float(val) if val is not None else None
        except (TypeError, ValueError):
            return None

    @property
    def turn(self) -> Optional[int]:
        val = self.payload.get("turn") or self.raw.get("turn")
        try:
            return int(val) if val is not None else None
        except (TypeError, ValueError):
            return None


In [ ]:
def load_long_horizon_events(path: Path) -> List[Event]:
    if not path.exists():
        raise FileNotFoundError(
            f"{path} not found.\n"
            "Please generate it first, for example:\n"
            "  python examples/synthetic_traces/generate_synthetic_traces.py \\\n"
            "    --variant long_horizon --sessions 5 --turns 40 \\\n"
            "    > examples/synthetic_traces/long_horizon_traces.jsonl"
        )
    events: List[Event] = []
    with path.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                obj = json.loads(line)
            except json.JSONDecodeError:
                continue
            events.append(Event(obj))
    return events

events = load_long_horizon_events(LONG_HORIZON_FILE)
traces = defaultdict(list)
for ev in events:
    traces[ev.trace_id].append(ev)
print(f"Loaded {len(events)} events across {len(traces)} sessions.")


In [ ]:
def plot_latency_curve(events: List[Event], title: str = ""):
    turns = []
    latencies = []
    drift_turns = []
    drift_lat = []
    corr_turns = []
    corr_lat = []

    for ev in events:
        t = ev.turn
        lat = ev.latency_ms
        if t is None or lat is None:
            continue
        turns.append(t)
        latencies.append(lat)
        if ev.event_type == "drift_like":
            drift_turns.append(t)
            drift_lat.append(lat)
        if ev.event_type in {"correction", "self_check"}:
            corr_turns.append(t)
            corr_lat.append(lat)

    if not turns:
        print("No latency-bearing events to plot.")
        return

    plt.figure(figsize=(8, 4))
    plt.plot(turns, latencies, marker="o")

    if drift_turns:
        plt.scatter(drift_turns, drift_lat, marker="x")
    if corr_turns:
        plt.scatter(corr_turns, corr_lat, marker="s")

    plt.xlabel("turn")
    plt.ylabel("latency_ms")
    plt.title(title or f"Trace {events[0].trace_id}")
    plt.tight_layout()
    plt.show()


In [ ]:
# Pick one trace with a drift/correction pattern (if present)
example_trace_id = None
for tid, evs in traces.items():
    kinds = {e.event_type for e in evs}
    if "drift_like" in kinds and ("correction" in kinds or "self_check" in kinds):
        example_trace_id = tid
        break

if example_trace_id is None:
    example_trace_id = next(iter(traces))

print(f"Using trace: {example_trace_id}")
plot_latency_curve(traces[example_trace_id], title=f"Latency curve for {example_trace_id}")
